# Sequence Generation across Different Models

In [1]:
import json
import pandas as pd
from src.models import *

In [2]:
with open("../Data/token_to_chord.json"):
    token_to_chord = json.load(open("../Data/token_to_chord.json"))

In [3]:
# The total number of sequences is the same as the size of the test sets
def generate_sequence_df(network, total_sequences=4448, batch_size=128):
    all_sequences = []
    for i in range(0, total_sequences, batch_size):
        size = min(batch_size, total_sequences - i) # Determine the current batch size
        sequences = network.generate_sequences(1.0, 254, size)
        all_sequences.extend(sequences)

    # Remove padding and convert to numpy
    all_sequences = [seq[1 : seq.argmax(-1)].cpu().numpy() for seq in all_sequences]

    return pd.DataFrame({"chords": all_sequences})

In [4]:
def save_df(df, file_name):
    df["chords"] = df["chords"].apply(lambda x: json.dumps(x.tolist()))
    df.to_csv(f"../Data/Generated/{file_name}.csv", index=False)

In [5]:
# Initialize all models
recurrent_net = recurrent_network.Main()

# Initialize models with variants
variants = ["small", "medium", "large"]
transformer_nets = [transformer.Main(variant) for variant in variants]
conditional_nets = [conditional_transformer.Main(variant) for variant in variants]
style_nets = [style_transformer.Main(variant) for variant in variants]

# Generate and save sequences for each model
models = [recurrent_net] + transformer_nets + conditional_nets + style_nets
model_names = ["recurrent_net"] + [f"transformer_{v}" for v in variants] + [f"conditional_{v}" for v in variants] + [f"style_{v}" for v in variants]

for model, name in zip(models, model_names):
    df = generate_sequence_df(model)
    save_df(df, name)

d:\Programs\anaconda3\lib\site-packages\torch\nn\modules\activation.py:1144: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\attention.cpp:152.)
  return torch._native_multi_head_attention(
